# TDMS -> WAV

In [1]:
import sys
import os

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# plt.style.use('seaborn-colorblind')
import datetime
from nptdms import TdmsFile
import wave

import matplotlib.pyplot as plt
sys.path.append('../')
import CaptureTracker as ct

### some convenient magics
%load_ext autoreload
%autoreload 2

In [2]:
datadir = '/Volumes/Aeroacoustics/_RawData/MicrophoneRecordings/'

In [4]:
dates = os.listdir(datadir)
rawdatafiles = os.listdir(os.path.join(datadir, dates[0]))
readpath = os.path.join(datadir,dates[0],rawdatafiles[0])

In [5]:
metadata = TdmsFile(readpath, read_metadata_only=True)
namethattune = TdmsFile(readpath)

In [6]:
df = namethattune.as_dataframe()

In [7]:
df.index = pd.TimedeltaIndex(data=1/51200*np.arange(len(df)), unit='s')# freq='195.3125U')

In [80]:
for cn in range(len(channelnames)):
    
    obj = wave.open('sound_channel_{}.wav'.format(channelnames[cn].split("'/'")[-1].split("'")[0]),'wb')

    sampleRate = 51200.0 # hertz
    sampleWidth = 2
    maxVolume = 2**(8*sampleWidth-1) - 1
    
    duration = np.round(df.shape[0]/51200, 2) # seconds

    obj.setnchannels(1) # mono
    obj.setsampwidth(sampleWidth)
    obj.setframerate(sampleRate)
        
    tmp = (maxVolume/2*df[channelnames[cn]]).astype(int).values
    
    formatlist = ['<'] + ['h']*len(tmp)
    formatlist = ('').join(formatlist)
    
    buf = struct.pack(formatlist, *tmp)
    obj.writeframesraw(buf)
    
    obj.close()

'TestNode_North9042_MicAA'

In [78]:
'sound_channel_{}.wav'.format(channelnames[0].split("'/'")[-1].split("'")[0])

'sound_channel_TestNode_North9042_MicAA.wav'